In [1]:
from FAdo.reex import *
from FAdo.cfg import *
from FAdo.fa import *
import cProfile
from os.path import commonprefix

In [2]:
# boolean that states if an atom is the first from the right child of a concatenation
def SupFirst(n):
    father = n.parent
    if father!=None and father.type == concat:
        if father.get_right()==n and not father.get_left().exp.ewp():
            return True
    return False

# boolean that states if an atom is the last from the left child of a concatenation
def SupLast(n):
    father = n.parent
    if father!=None and father.type == concat:
        if father.get_left()==n and not father.get_right().exp.ewp():
            return True
    return False

# pointer to the first(right child) of a concatenation
def pSupFirst(n):
    while not SupFirst(n):
        if len(n.path) == 1:
            return None
        n = n.parent
    return n

# pointer to the last(left child) of a concatenation
def pSupLast(n):
    while not SupLast(n):
        if len(n.path) == 1:
            return None
        n = n.parent
    return n
        

In [3]:
# lemma 2.3
def isFirst(n,p):
    if p.type != position or n.type == position:
        return False
    k = pSupFirst(p)
    if k != None:
        if p.reflexive(n) and n.reflexive(k):
            return True
    #"""
    k = p.LSA()
    if k != None:
        if p.reflexive(n) and n.reflexive(k):
            return True
    #"""
    return False

def isLast(n,p):
    if p.type != position or n.type == position:
        return False
    k = pSupLast(p)
    if k != None:
        if p.reflexive(n) and n.reflexive(k):
            return True
        #"""
    k = p.LSA()
    if k != None :
        if p.reflexive(n) and n.reflexive(k):
            return True
            #"""
    return False

In [4]:
class eTree:
    def __init__(self,regex,path):
        self.NT = dict()
        self.root = construct(regex,path,self)
        return
    
    def followList(self):
        l = dict()
        atoms = set()
        for x in self.NT:
            a = self.NT.get(x)
            if a.type==position:
                atoms.add(a)
        for x in atoms:
            f_list = set()
            for w in atoms:
                if w.type!=epsilon:
                    if w.follow(x):
                        f_list.add(w.exp)
            l[x.exp]=f_list
            #print x.exp,f_list
        return l

def construct(reg_exp,path,t):
        node = Node(reg_exp,path,t)
        t.NT[node.path] = node
        if node.type==star:
            construct(node.exp.arg,path+'1',t)
        elif node.type==concat or node.type==disj:
            construct(node.exp.arg1,path+'1',t)
            construct(node.exp.arg2,path+'2',t)
        return node

In [11]:
class Node:
    
    def __init__(self,expr,string,t):
        self.type  = type(expr)
        self.path  = string
        self.exp   = expr
        self.tree  = t
        self.right = None
        self.left  = None
        self.first = None
        self.last  = None
        self.lsa   = None
        if self.type == position:
            self.posix = expr.val[1]
        if string == '1':
            self.parent = None
        else:
            self.parent = t.NT.get(string[:-1])

    def get_left(self):
        if self.left==None:
            self.left = self.tree.NT.get(self.path+'1')
        return self.left
    
    def get_right(self):
        if self.right==None:
            self.right = self.tree.NT.get(self.path+'2')
        return self.right
        
    def get_root(self):
        return self.t.root
    
    def get_brother(self):
        if self.path[len(self.path)-1]==1:
            node = self.parent.get_right()
        else:
            node = self.parent.get_left()
        return node
    
    def reflexive(self,n):
        if n == None:
            return False
        if commonprefix([self.path,n.path])==n.path: #LCA(self,n)
            return True
        else:
            return False    
    
    def llast(self):
        if self == None:
            return set()
        if self.last != None:
            return self.last
        self.last = set()
        if self.type == star:
            self.last = self.get_left().llast()
            return self.last
        if self.type == concat:
            if self.get_right().exp.ewp():
                left = self.get_left().llast()
                self.last = self.last.union(left)
                right = self.get_right().llast()
                self.last = self.last.union(right)
                return self.last
            self.last = self.get_right().llast()
            return self.last
        if self.type == disj:
            left = self.get_left().llast()
            self.last = self.last.union(left)
            right = self.get_right().llast()
            self.last = self.last.union(right)
            return self.last
        if self.type == position:
            self.last = set([self.exp])
        return self.last
    
    def ffirst(self):
        if self == None:
            return set()
        if self.first != None:
            return self.first
        self.first = set()
        if self.type == star:
            self.first = self.get_left().ffirst()
            return self.first
        if self.type == concat:
            if self.get_left().exp.ewp():
                left = self.get_left().ffirst()
                self.first = self.first.union(left)
                right = self.get_right().ffirst()
                self.first = self.first.union(right)
                return self.first
            self.first = self.get_left().ffirst()
            return self.first
        if self.type == disj:
            left = self.get_left().ffirst()
            self.first = self.first.union(left)
            right = self.get_right().ffirst()
            self.first = self.first.union(right)
            return self.first
        if self.type == position:
            self.first = set([self.exp])
        return self.first
    
    def follow(self,p):
        node1 = self.tree.NT.get(commonprefix([p.path,self.path]))  # LCA(p,self)
        if node1 == None:
            return False
        if node1.type==concat:
            if set([self.exp]).issubset(node1.get_right().ffirst()):
                if set([p.exp]).issubset(node1.get_left().llast()):
                    return True
        node = node1.LSA()
        if node == None:
            return False
        if self.exp in node.ffirst() and p.exp in node.llast():
            return True
        return False
    
    def LSA(self):
        if self.lsa != None:
            return self.lsa
        if self.type == star:
            self.lsa = self
        elif len(self.path)==1:
            self.lsa = None
        else:
            self.lsa = self.parent.LSA()
        return self.lsa

# Last Common Ancestor
#def LCA(p,q):
#    return commonprefix([p.path,q.path])

In [18]:
op = reStringRGenerator(Sigma=['a'], size=500, cfgr=reGrammar["g_rpn"])
%time a1 = op.generate()
%time b1 = str2regexp(a1,parser=ParserRPN)
print "RegEx generated!"
b2 = b1.marked()
print "RegEx marked! Waitting for tree construction...."
t = eTree(b2,'1')
print "Done!"
i_win()

CPU times: user 15.8 ms, sys: 0 ns, total: 15.8 ms
Wall time: 15.8 ms
CPU times: user 18 ms, sys: 137 µs, total: 18.1 ms
Wall time: 17.8 ms
RegEx generated!
RegEx marked! Waitting for tree construction....
Done!
I GOT:
1 loop, best of 3: 225 ms per loop

They have:
100 loops, best of 3: 14 ms per loop


In [17]:
def i_win():
    print "I GOT:"
    %timeit t.followList()
    print
    print "They have:"
    %timeit b2.followListsD()

In [15]:
#%time t.followList()
cProfile.run("t.followList()")
cProfile.run("b2.followListsD()")

         214492 function calls (191927 primitive calls) in 0.358 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10816    0.107    0.000    0.337    0.000 <ipython-input-11-98d6e069026e>:104(follow)
    10095    0.018    0.000    0.018    0.000 <ipython-input-11-98d6e069026e>:119(LSA)
     1404    0.003    0.000    0.003    0.000 <ipython-input-11-98d6e069026e>:20(get_left)
     5808    0.012    0.000    0.012    0.000 <ipython-input-11-98d6e069026e>:25(get_right)
     4719    0.010    0.000    0.010    0.000 <ipython-input-11-98d6e069026e>:48(llast)
    15903    0.029    0.000    0.029    0.000 <ipython-input-11-98d6e069026e>:76(ffirst)
        1    0.013    0.013    0.358    0.358 <ipython-input-4-2942987aec85>:7(followList)
        1    0.000    0.000    0.358    0.358 <string>:1(<module>)
    10816    0.069    0.000    0.078    0.000 genericpath.py:76(commonprefix)
    22565    0.020    0.000    0.047    0.000 reex.

In [10]:
%timeit t.followList()

1 loop, best of 3: 802 ms per loop


In [11]:
%timeit _=b2.followListsD()

100 loops, best of 3: 7.16 ms per loop


In [13]:
%timeit _=t.followList()

1 loop, best of 3: 2.71 s per loop


In [ ]:
def last_cat(n):
    if n.type == concat:
        return n
    if len(n.path) == 1:
        return None
    return last_cat(n.parent)
    
def buildNext(a,n,y):
        if pSupLast(n):
            y = set()
        c = last_cat(n)
        n2 = n.get_brother()
        if c != None and n.reflexive(c.get_left()) and n2!=None and (not SupLast(n) or n!=c):
            y.add() #terminar....
        

In [50]:
commonprefix(["12345","1257"])

'12'